# Project 5: Feature Selection and LASSO

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO. You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.
In the second part, you will implement your own LASSO solver, using coordinate descent.

We will continue to use the House data from Redfin.

In [104]:
import numpy as np
import pandas as pd
import pprint

In [105]:
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.preprocessing import scale

# Load in house sales data

In [106]:
# your code
df = pd.read_csv('santa_clara_county.csv')
df = df[df['PRICE'].notnull()]
df = df[df['SQUARE FEET'].notnull()]
df = df[df['LOT SIZE'].notnull()]
df = df[df['BEDS'].notnull()]
df = df[df['BATHS'].notnull()]
df = df[df['ZIP OR POSTAL CODE'].notnull()]
df = df[df['YEAR BUILT'].notnull()]
df = df[df['CITY'].notnull()]
df = df[df['PROPERTY TYPE'].notnull()]

# Create new features

Recall in Project 2, we consider features that are some transformations of inputs.
Create square root of SQUARE FEET, square root of LOT SIZE, SQUARE of BEDS, and SQUARE of BATHS
* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.


In [107]:
# your code
import math
from math import sqrt

In [108]:
df['sqft_sqrt'] = df['SQUARE FEET'].apply(sqrt)
df['lot_sqrt']= df['LOT SIZE'].apply(sqrt)
df['bedrooms_square'] = df['BEDS'].apply(lambda x:x**2)
df['bath_square'] = df['BATHS'].apply(lambda x:x**2)

# Learn regression weights with L1 penalty


Let us fit a model with all the features available ('SQUARE FEET', 'LOT SIZE', 
                'BEDS', 'BATHS', 'CITY', 
                'PROPERTY TYPE', 'ZIP OR POSTAL CODE', 
                'YEAR BUILT'), plus the features we just created above. 
                
Note: For 'CITY' and 'PROPERTY TYPE' convert them to categorical features first then do label encoding.

In [109]:
df['CITY'] = df['CITY'].astype('category')
df['PROPERTY TYPE'] = df['PROPERTY TYPE'].astype('category')

In [110]:
df['CITY_cat'] = df['CITY'].cat.codes
df['PROPERTY TYPE_cat'] = df['PROPERTY TYPE'].cat.codes

In [111]:
all_features = ['SQUARE FEET', 'LOT SIZE', 
                'BEDS', 'BATHS', 'CITY_cat', 
                'PROPERTY TYPE_cat', 'ZIP OR POSTAL CODE', 
                'YEAR BUILT', 'sqft_sqrt','lot_sqrt','bedrooms_square','bath_square']

Applying L1 penalty = 1e7 using lasso Regression.

In [112]:
# your code
L1_penalty = 1e7
model = Lasso(alpha = L1_penalty)
lasso_model = model.fit(df[all_features],df['PRICE'])

Find what features had non-zero weight.

In [113]:
# copy over the function which prints the coefficients and intercept.
def coef(coefficients,intercept,input_features):
    n = len(input_features)
    for i in range(0,n):
        print ('The coefficient for ' + input_features[i] + ' is ' + str(coefficients[i]))
    print ('The intercept is ' + str(intercept))

In [114]:
# your code 
coef(lasso_model.coef_,lasso_model.intercept_,all_features)

The coefficient for SQUARE FEET is 729.0099588410328
The coefficient for LOT SIZE is 4.138578724255458
The coefficient for BEDS is 0.0
The coefficient for BATHS is -0.0
The coefficient for CITY_cat is -0.0
The coefficient for PROPERTY TYPE_cat is 0.0
The coefficient for ZIP OR POSTAL CODE is -0.0
The coefficient for YEAR BUILT is -0.0
The coefficient for sqft_sqrt is -0.0
The coefficient for lot_sqrt is 0.0
The coefficient for bedrooms_square is 0.0
The coefficient for bath_square is -0.0
The intercept is 27692.56800360931


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection.

According to this list of weights, which of the features have been chosen?

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets with 45/45/10

In [115]:
# your code
train_set = df.sample(frac= 0.45, random_state = 1)
rf = df.drop(train_set.index)
validation_set = rf.sample(frac = 0.45, random_state = 1)
test_set = df.drop(validation_set.index)

Next, we write a loop that does the following:
For l1_penalty in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^8] (to get this in Python, type np.logspace(1, 8, num=15).)
Fit a regression model with a given l1_penalty on TRAIN data. Specify alpha=l1_penalty in the parameter list.
Compute the RSS on VALIDATION data for that l1_penalty
Report which l1_penalty produced the lowest RSS on validation data.

In [122]:
validation_rss = {}
for l1_penalty in np.logspace(1,8,num=15):
    
    model = Lasso(alpha = l1_penalty)
    
    lasso_model = model.fit(train_set[all_features],train_set['PRICE'])
    predictions = model.predict(validation_set[all_features])
    
    residuals = validation_set['PRICE'] - predictions
    rss = sum(residuals**2)
    validation_rss[l1_penalty] = rss
    
pprint.pprint (validation_rss)

{10.0: 58575498910376.41,
 31.622776601683793: 58562343424770.13,
 100.0: 58520659471631.94,
 316.22776601683796: 58388184793617.125,
 1000.0: 57973111441527.21,
 3162.2776601683795: 56771185203116.84,
 10000.0: 53601926732321.86,
 31622.776601683792: 47989620805568.67,
 100000.0: 50739454518719.82,
 316227.7660168379: 58643552324142.87,
 1000000.0: 59269440702711.47,
 3162277.6601683795: 58675488017651.48,
 10000000.0: 58639777402570.07,
 31622776.60168379: 58756537893387.12,
 100000000.0: 59268720710420.05}


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of

In [87]:
print (min(validation_rss.items(), key = lambda x:x[1]))

(31622.776601683792, 47989620805568.67)


What was the best value for the l1_penalty? and use this value of L1 penalty, how many nonzero weights do you have?

In [124]:
# your code
L1_penalty = 31622.776601683792
model = Lasso(alpha = L1_penalty)
lasso_model = model.fit(train_set[all_features],train_set['PRICE'])
coef(lasso_model.coef_,lasso_model.intercept_,all_features)

The coefficient for SQUARE FEET is 2721.8753171803296
The coefficient for LOT SIZE is 80.30452076552201
The coefficient for BEDS is 0.0
The coefficient for BATHS is 0.0
The coefficient for CITY_cat is -1086625.5608474899
The coefficient for PROPERTY TYPE_cat is -275993.59279605834
The coefficient for ZIP OR POSTAL CODE is 9643.911388233591
The coefficient for YEAR BUILT is -1546.1582516172948
The coefficient for sqft_sqrt is -200742.13359684445
The coefficient for lot_sqrt is 5134.41220380192
The coefficient for bedrooms_square is 47489.29374238888
The coefficient for bath_square is -50808.88870658876
The intercept is -898059084.9188739


# Limit the number of nonzero weights

In data industry, sometimes there are limitations to derive What if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them. Now let's consider the case where if we absolutely wanted to limit ourselves to, say, 7 features. 

In this section, you are going to implement a simple, two phase procedure to achive this goal:
Explore a large range of l1_penalty values to find a narrow region of l1_penalty values where models are likely to have the desired number of non-zero weights.
Further explore the narrow region you found to find a good value for l1_penalty that achieves the desired sparsity. Here, we will again use a validation set to choose the best value for l1_penalty.

In [89]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible l1_penalty_values:

In [90]:
l1_penalty_values = np.logspace(5, 6, num=20)

Now, implement a loop that search through this space of possible l1_penalty values:
For l1_penalty in np.logspace(5, 6, num=20):
Fit a regression model with a given l1_penalty on TRAIN data. Specify alpha=l1_penalty. in the parameter list.
Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.


In [121]:
# your code
weight_coef = {}
for l1_penalty in l1_penalty_values:
    model = Lasso(alpha = l1_penalty)
    lasso_model = model.fit(train_set[all_features],train_set['PRICE'])
    weight_coef[l1_penalty] = np.count_nonzero(lasso_model.coef_)
pprint.pprint (l1_penalty,np.count_nonzero(lasso_model.coef_))

{100000.0: (array([ 0,  1,  4,  6,  7,  8,  9, 10, 11]),),
 112883.78916846884: (array([ 0,  1,  4,  6,  7,  8,  9, 10, 11]),),
 127427.49857031347: (array([ 0,  1,  4,  6,  7,  8,  9, 10, 11]),),
 143844.9888287663: (array([ 0,  1,  4,  6,  7,  8,  9, 10, 11]),),
 162377.67391887208: (array([ 0,  1,  4,  6,  7,  8,  9, 10, 11]),),
 183298.07108324376: (array([ 0,  1,  6,  7,  8,  9, 10, 11]),),
 206913.808111479: (array([ 0,  1,  6,  7,  8,  9, 10, 11]),),
 233572.14690901214: (array([ 0,  1,  6,  7,  8,  9, 10, 11]),),
 263665.08987303555: (array([ 0,  1,  6,  7,  8,  9, 10, 11]),),
 297635.14416313195: (array([ 0,  1,  6,  7,  8,  9, 10, 11]),),
 335981.8286283781: (array([ 0,  1,  6,  7,  8,  9, 10, 11]),),
 379269.0190732254: (array([ 0,  1,  6,  7,  8,  9, 10, 11]),),
 428133.2398719396: (array([ 0,  1,  6,  7,  9, 10, 11]),),
 483293.0238571752: (array([ 0,  1,  6,  7,  9, 10, 11]),),
 545559.4781168514: (array([ 0,  1,  6,  7, 10, 11]),),
 615848.2110660267: (array([ 0,  1,  6,

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, can you try to code it up with at least two ways:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [96]:
# your code

183298.071083
545559.478117


What values did you find for l1_penalty_min and l1_penalty_max, respectively?

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [97]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `alpah =l1_penalty` in the parameter list. 
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [98]:
# your code
validation_rss = {}
for l1_penalty in l1_penalty_values:
    model = Lasso(alpha = L1_penalty)
    lasso_model = model.fit(train_set[all_features],train_set['PRICE'])
    predictions = model.predict(validation_set[all_features])
    residuals = validation_set['PRICE'] -predictions
    rss = sum(residuals**2)
    validation_rss[l1_penalty] = np.count_nonzero(lasso_model.coef_)
pprint.pprint (validation_rss,rss)

(183298.07108324376, 8, 1842110709172621.0)
(202364.46092711785, 7, 1763625707853340.2)
(221430.85077099194, 7, 1719653458102102.5)
(240497.24061486602, 7, 1676259035664565.5)
(259563.63045874011, 7, 1633426409860284.8)
(278630.0203026142, 7, 1591167401829669.8)
(297696.41014648828, 7, 1549399714751410.8)
(316762.79999036237, 7, 1508131310832209.8)
(335829.18983423646, 7, 1470458351324287.8)
(354895.57967811055, 7, 1431154947209693.2)
(373961.96952198463, 7, 1392399914103245.2)
(393028.35936585872, 7, 1354171875124205.0)
(412094.74920973281, 7, 1316365422496779.0)
(431161.13905360689, 7, 1279183704126268.2)
(450227.52889748098, 7, 1242712178804889.0)
(469293.91874135507, 7, 1206843788883919.8)
(488360.30858522916, 7, 1171581193967222.0)
(507426.69842910324, 6, 1128195528575776.8)
(526493.08827297739, 6, 1073388880271762.6)
(545559.47811685142, 6, 1016473953078559.2)


You can try another round of fine tuning of the parameters. The main technique to learn here is how to tune the parameters using granular way as described here.

What value of l1_penalty in our narrow range has the lowest RSS on the VALIDATION set and has sparsity equal to max_nonzeros?

What features in this model have non-zero coefficients?

In [101]:
# your code
L1_penalty = 
model = Lasso(alpha = L1_penalty)
lasso_model = model.fit(train_set[all_features],train_set['PRICE'])
coef(lasso_model.coef_,lasso_model.intercept_,all_features)

The coefficient for SQUARE FEET is 942.580193609
The coefficient for LOT SIZE is 261.778092592
The coefficient for BEDS is -0.0
The coefficient for BATHS is -0.0
The coefficient for CITY_cat is -0.0
The coefficient for PROPERTY TYPE_cat is -0.0
The coefficient for ZIP OR POSTAL CODE is -2022.20098513
The coefficient for YEAR BUILT is 1899.88039708
The coefficient for sqft_sqrt is -4540.26498027
The coefficient for lot_sqrt is -19832.7645654
The coefficient for bedrooms_square is -0.0
The coefficient for bath_square is -34158.357294
The intercept is 186952695.967


Does the non-zero coefficient makes sense to you? what does each of the parametere mean?

# Coordinate Descent for Lasso Regression

In this section, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

Let's reuse the get_num_data() and predict_output() function from Project 2. Copy them over.

In [125]:
# your code
def get_numpy_data(data, features, output):
    data['constant'] = 1
    features = ['constant'] + features
    feature_frame = data[features]
    feature_matrix = feature_frame.to_numpy()
    output_array = data[output].to_numpy()
    return(feature_matrix, output_array)

In [126]:
# your code
def predict_output(feature_matrix, weights):
    predictions = np.dot(feature_matrix,weights)
    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `SQUARE FEET` is very large overall compared to `BEDS`, for instance. As a result, weight for `SQUARE FEET` would be much smaller than weight for `BEDS`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features**: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [128]:
X = np.array([[1.,3.,9.],[4.,12.,19.]])
print (X)

[[ 1.  3.  9.]
 [ 4. 12. 19.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [130]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print (norms)

[ 4.12310563 12.36931688 21.02379604]


To normalize, apply element-wise division:

In [131]:
print (X / norms) # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[0.24253563 0.24253563 0.42808634]
 [0.9701425  0.9701425  0.90373784]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. We will use these norms to normalize the test data in the same way as we normalized the training data. 

In [132]:
# your code
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis = 0)
    normalized_features = feature_matrix / norms
    return(normalized_features, norms)

To test your function, run the following:

In [134]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print (features)
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print (norms)
# should print
# [5.  10.  15.]

[[0.6 0.6 0.6]
 [0.8 0.8 0.8]]
[ 5. 10. 15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [135]:
simple_features = ['SQUARE FEET', 'BEDS']
my_output = 'PRICE'
(simple_feature_matrix, output) = get_numpy_data(df, simple_features, my_output)

Normalize the features first:

In [136]:
# your code
simple_feature_matrix,norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of ro[i]:

In [137]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [142]:
# your code
prediction = predict_output(simple_feature_matrix, weights)

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [146]:
# your code
ro = [0 for h in range((simple_feature_matrix.shape)[1])]
for i in range ((simple_feature_matrix.shape)[1]):
    ro[i] = ((simple_feature_matrix[:,i])*(output - prediction + weights[i]*simple_feature_matrix[:,i])).sum()
print (ro)

[19614517.909663543, 21803049.009274505, 21091484.864712715]


Whenever ro[i] falls between -l1_penalty/2 and l1_penalty/2, the corresponding weight w[i] is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of l1_penalty would not set w[1] zero, but would set w[2] to zero, if we were to take a step in that coordinate?

In [120]:
# your code

-42182969.7294
-43606098.0185


What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

In [121]:
# your code

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [147]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    # your code
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    # your code
    ro_i= ((feature_matrix[:,i])*(output - prediction + weights[i]*feature_matrix[:,i])).sum()
    if i == 0: # intercept -- do not regularize
        # your code 
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i+l1_penalty/2.
    elif ro_i > -l1_penalty/2.:
        new_weight_i = ro_i-l1_penalty/2.
    else:
        # your code
        new_weight_i = 0
    
    return new_weight_i

To test the function, run the following cell:

In [149]:
# should print 0.425558846691
import math
print (lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.4255588466910251


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [125]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    # your code #initialize weights
    D = feature_matrix.shape[1]
    weights = np.array(initial_weights)
    change_in_coordination = tolerance * np.ones(len(weights)) # define change_in_coordination
    while max(change_in_coordination) >= tolerance: 
        #for loop
        # your code
        for i in range(D):
            old_weight_i = weights[i]
            new_weight_i = lasso_coordinate_descent_step(i,feature_matrix,output,weights,l1_penalty)
            change_in_coordination = np.abs(new_weight_i-old_weight_i)
            weights[i] = new_weight_i
    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [133]:
simple_features = ['SQUARE FEET', 'BEDS']
my_output = 'PRICE'
initial_weights = np.zeros(3)
l1_penalty = 5e6
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [134]:
# your code
# normalize features
(simple_feature_matrix, output)=get_numpy_data(df,simple_features,my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix)

Then, run your implementation of LASSO coordinate descent:

In [135]:
# your code
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print (weights)

[ 11755239.08267321   8777351.68323666         0.        ]


***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

In [136]:
# your code
prediction =  predict_output(normalized_simple_feature_matrix, weights)
RSS = np.dot(output-prediction, output-prediction)
print (RSS)

2.87204485517e+14


# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets using 80/20

In [138]:
# your code
train_data= df.sample(frac= 0.8, random_state = 1)
test_data = df.drop(train_data.index)

Let's use all features as used previously.

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [140]:
# your code
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
normalized_feature_matrix, norms = normalize_features(feature_matrix)

First, learn the weights with `l1_penalty=1e6`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e6`, you will need them later.

In [ ]:
# your code
initial_weights = np.zeros(len(all_features) + 1)
l1_penalty = 1e6
tolerance = 1.0

In [146]:
weights1e6 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                               initial_weights, l1_penalty, tolerance)
print (weights1e6)

[  3427219.69506716  15543958.67698757    901906.58096379         0.
         0.          -2254389.36321642         0.                 0.
         0.                 0.                 0.           1093835.60511549
    711968.73621285]


What features had non-zero weight in this case?

Next, learn the weights with `l1_penalty=5e6`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights5e6`, you will need them later.

In [148]:
# your code
initial_weights = np.zeros(len(all_features) + 1)
l1_penalty = 5e6
tolerance = 1.0
weights5e6 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                               initial_weights, l1_penalty, tolerance)
print (weights5e6)

[ 12000240.56346168   2339995.9245499          0.                 0.
         0.                 0.                 0.                 0.
         0.                 0.                 0.                 0.
   4350532.75068793]


What features had non-zero weight in this case?

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=1e2`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [186]:
# your code
initial_weights = np.zeros(len(all_features) + 1)
l1_penalty = 1e4
tolerance = 1e2
weights1e4 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                               initial_weights, l1_penalty, tolerance)
print (weights1e4)

[  8.38339161e+07   7.21034247e+07   7.42341068e+06  -2.97719236e+06
   9.71342669e+06  -2.15449552e+07   1.49616373e+06   0.00000000e+00
  -1.65440163e+07  -1.10956675e+08   0.00000000e+00   4.66340738e+06
  -4.68195860e+06]


What features had non-zero weight in this case?

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e6`, `weights5e6`, `weights1e4`).

In [189]:
# your code
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
normalized_feature_matrix, norms = normalize_features(feature_matrix)

In [ ]:
normalized_weights1e6 = weights1e6 / norms
normalized_weights5e6 = weights5e6 / norms
normalized_weights1e4 = weights1e4 / norms

To check your results, if you call `normalized_weights1e6` the normalized version of `weights1e6`, should return 11.22951655151892

In [192]:
weights1e6_normalized[2]

11.22951655151892

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [195]:
# your code
(test_feature_matrix, test_output)=get_numpy_data(test_data, all_features,'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [197]:
# your code
prediction = predict_output(test_feature_matrix, normalized_weight1e6)
RSS1 = np.dot(test_output-prediction,test_output-prediction)
prediction = predict_output(test_feature_matrix, normalized_weight5e6)
RSS2 = np.dot(test_output-prediction,test_output-prediction)
prediction = predict_output(test_feature_matrix, normalized_weight1e4)
RSS3 = np.dot(test_output-prediction,test_output-prediction)
PRINT (RSS1,RSS2,RSS3)

6.97979110651e+13
7.78205230466e+13
3.05852922918e+14


Which model performed best on the test data? What is your interpretation on the results?

In [ ]:
first model performed best on the test data